# Bygg en regresjonsmodell: forbered og visualiser data

## **Lineær regresjon for gresskar - Leksjon 2**
#### Introduksjon

Nå som du har verktøyene du trenger for å begynne å bygge maskinlæringsmodeller med Tidymodels og Tidyverse, er du klar til å begynne å stille spørsmål til dataene dine. Når du jobber med data og bruker ML-løsninger, er det svært viktig å forstå hvordan du stiller de riktige spørsmålene for å virkelig utnytte potensialet i datasettet ditt.

I denne leksjonen vil du lære:

- Hvordan forberede dataene dine for modellbygging.

- Hvordan bruke `ggplot2` til datavisualisering.

Spørsmålet du ønsker svar på, vil avgjøre hvilken type ML-algoritmer du vil bruke. Og kvaliteten på svaret du får, vil i stor grad avhenge av kvaliteten på dataene dine.

La oss se på dette gjennom en praktisk øvelse.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Kunstverk av @allison_horst</figcaption>


<!--![Kunstverk av \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Kunstverk av \@allison_horst-->


## 1. Importere data om gresskar og tilkalle Tidyverse

Vi trenger følgende pakker for å bearbeide og analysere denne leksjonen:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) er en [samling av R-pakker](https://www.tidyverse.org/packages) som er laget for å gjøre datavitenskap raskere, enklere og morsommere!

Du kan installere dem slik:

`install.packages(c("tidyverse"))`

Skriptet nedenfor sjekker om du har de nødvendige pakkene for å fullføre dette modulen, og installerer dem for deg hvis noen mangler.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Nå, la oss fyre opp noen pakker og laste inn [data](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) som er gitt for denne leksjonen!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

En rask `glimpse()` viser umiddelbart at det finnes tomme verdier og en blanding av strenger (`chr`) og numeriske data (`dbl`). `Date` er av typen karakter, og det finnes også en merkelig kolonne kalt `Package` hvor dataene er en blanding av `sacks`, `bins` og andre verdier. Dataene er, ærlig talt, litt av et rot 😤.

Det er faktisk ikke veldig vanlig å få et datasett som er helt klart til bruk for å lage en ML-modell rett ut av boksen. Men fortvil ikke, i denne leksjonen vil du lære hvordan du kan klargjøre et rådatasett ved hjelp av standard R-biblioteker 🧑‍🔧. Du vil også lære ulike teknikker for å visualisere dataene. 📈📊
<br>

> En oppfriskning: Pipe-operatoren (`%>%`) utfører operasjoner i logisk rekkefølge ved å sende et objekt videre inn i en funksjon eller et uttrykk. Du kan tenke på pipe-operatoren som å si "og så" i koden din.


## 2. Sjekk etter manglende data

En av de vanligste utfordringene dataforskere må håndtere er ufullstendige eller manglende data. R representerer manglende eller ukjente verdier med en spesiell sentinel-verdi: `NA` (Not Available).

Så hvordan kan vi finne ut om datasettet inneholder manglende verdier?  
<br>
-   En enkel måte er å bruke base R-funksjonen `anyNA`, som returnerer de logiske verdiene `TRUE` eller `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Flott, det ser ut til å mangle noen data! Det er et godt sted å starte.

-   En annen måte er å bruke funksjonen `is.na()` som viser hvilke individuelle kolonneelementer som mangler ved å returnere en logisk `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Ok, jobben er gjort, men med en så stor dataramme som dette, ville det vært ineffektivt og praktisk talt umulig å gå gjennom alle radene og kolonnene individuelt😴.

-   En mer intuitiv måte ville være å beregne summen av de manglende verdiene for hver kolonne:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Mye bedre! Det mangler data, men kanskje det ikke vil ha betydning for oppgaven. La oss se hva videre analyse bringer frem.

> Sammen med de fantastiske pakkene og funksjonene, har R veldig god dokumentasjon. For eksempel, bruk `help(colSums)` eller `?colSums` for å finne ut mer om funksjonen.


## 3. Dplyr: En grammatikk for datamanipulering

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Illustrasjon av @allison_horst</figcaption>


<!--![Illustrasjon av \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Illustrasjon av \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), en pakke i Tidyverse, er en grammatikk for datamanipulering som tilbyr et konsistent sett med verb som hjelper deg med å løse de vanligste utfordringene innen datamanipulering. I denne delen skal vi utforske noen av dplyr sine verb!
<br>


#### dplyr::select()

`select()` er en funksjon i pakken `dplyr` som hjelper deg med å velge kolonner du vil beholde eller utelate.

For å gjøre datarammen din enklere å jobbe med, fjern flere av kolonnene ved å bruke `select()`, og behold kun de kolonnene du trenger.

For eksempel, i denne øvelsen vil analysen vår involvere kolonnene `Package`, `Low Price`, `High Price` og `Date`. La oss velge disse kolonnene.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` er en funksjon i pakken `dplyr` som hjelper deg med å lage eller endre kolonner, samtidig som de eksisterende kolonnene beholdes.

Den generelle strukturen for mutate er:

`data %>%   mutate(new_column_name = what_it_contains)`

La oss prøve `mutate` ved å bruke `Date`-kolonnen og utføre følgende operasjoner:

1. Konverter datoene (som for øyeblikket er av typen karakter) til et månedsformat (disse er amerikanske datoer, så formatet er `MM/DD/YYYY`).

2. Ekstraher måneden fra datoene til en ny kolonne.

I R gjør pakken [lubridate](https://lubridate.tidyverse.org/) det enklere å jobbe med dato- og tidsdata. Så la oss bruke `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` og se hvordan vi kan oppnå de ovennevnte målene. Vi kan fjerne Date-kolonnen siden vi ikke vil trenge den igjen i senere operasjoner.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Woohoo! 🤩

La oss nå opprette en ny kolonne `Price`, som representerer gjennomsnittsprisen på et gresskar. Nå skal vi ta gjennomsnittet av kolonnene `Low Price` og `High Price` for å fylle ut den nye kolonnen Price.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Jaaa!💪

"Men vent litt!", sier du etter å ha skummet gjennom hele datasettet med `View(pumpkins)`, "Det er noe rart her!"🤔

Hvis du ser på `Package`-kolonnen, blir gresskar solgt i mange forskjellige konfigurasjoner. Noen selges i `1 1/9 bushel`-mål, noen i `1/2 bushel`-mål, noen per gresskar, noen per pund, og noen i store bokser med varierende bredder.

La oss bekrefte dette:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Fantastisk!👏

Gresskar ser ut til å være veldig vanskelige å veie konsekvent, så la oss filtrere dem ved å velge kun gresskar med strengen *bushel* i `Package`-kolonnen og legge dette i en ny data frame `new_pumpkins`.


#### dplyr::filter() og stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): lager et delsett av dataene som kun inneholder **rader** som oppfyller dine betingelser, i dette tilfellet gresskar med strengen *bushel* i `Package`-kolonnen.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): oppdager tilstedeværelsen eller fraværet av et mønster i en streng.

[`stringr`](https://github.com/tidyverse/stringr)-pakken gir enkle funksjoner for vanlige operasjoner med strenger.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Du kan se at vi har snevret det ned til rundt 415 rader med data som inneholder gresskar i store mengder.🤩  
<br>


#### dplyr::case_when()

**Men vent! Det er én ting til å gjøre**

La du merke til at mengden per skjeppe varierer per rad? Du må normalisere prisen slik at du viser prisen per skjeppe, ikke per 1 1/9 eller 1/2 skjeppe. Tid for litt matte for å standardisere det.

Vi skal bruke funksjonen [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) for å *mutere* kolonnen Price avhengig av noen betingelser. `case_when` lar deg vektorisere flere `if_else()`-utsagn.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Nå kan vi analysere prisen per enhet basert på deres måling i skjepper. All denne studien av skjepper med gresskar viser imidlertid hvor `viktig` det er å `forstå naturen til dataene dine`!

> ✅ Ifølge [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308) avhenger vekten av en skjepp av typen råvare, siden det er en volum-måling. "En skjepp med tomater, for eksempel, skal veie 56 pund... Blader og grønnsaker tar opp mer plass med mindre vekt, så en skjepp med spinat veier bare 20 pund." Det er ganske komplisert! La oss ikke bry oss med å gjøre en skjepp-til-pund-konvertering, og heller prise per skjepp. All denne studien av skjepper med gresskar viser imidlertid hvor viktig det er å forstå naturen til dataene dine!
>
> ✅ La du merke til at gresskar som selges per halv skjepp er veldig dyre? Kan du finne ut hvorfor? Hint: små gresskar er mye dyrere enn store, sannsynligvis fordi det er så mange flere av dem per skjepp, gitt den ubrukte plassen som tas opp av ett stort, hul paigresskar.


Nå til slutt, for eventyrets skyld 💁‍♀️, la oss også flytte Måned-kolonnen til første posisjon, altså `før` kolonnen `Pakke`.

`dplyr::relocate()` brukes for å endre kolonneposisjoner.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Godt jobbet!👌 Du har nå et rent og ryddig datasett som du kan bruke til å bygge din nye regresjonsmodell!


## 4. Datavisualisering med ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografikk av Dasani Madipalli</figcaption>


<!--![Infografikk av Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Det finnes et *klokt* ordtak som sier:

> "Den enkle grafen har brakt mer informasjon til dataanalytikerens sinn enn noe annet verktøy." --- John Tukey

En del av dataforskerens rolle er å demonstrere kvaliteten og naturen til dataene de jobber med. For å gjøre dette lager de ofte interessante visualiseringer, eller diagrammer, grafer og figurer, som viser ulike aspekter av dataene. På denne måten kan de visuelt vise sammenhenger og mangler som ellers er vanskelige å oppdage.

Visualiseringer kan også hjelpe med å avgjøre hvilken maskinlæringsteknikk som er mest passende for dataene. Et spredningsdiagram som ser ut til å følge en linje, for eksempel, indikerer at dataene er en god kandidat for en lineær regresjonsøvelse.

R tilbyr flere systemer for å lage grafer, men [`ggplot2`](https://ggplot2.tidyverse.org/index.html) er en av de mest elegante og allsidige. `ggplot2` lar deg sette sammen grafer ved å **kombinere uavhengige komponenter**.

La oss starte med et enkelt spredningsdiagram for kolonnene Price og Month.

I dette tilfellet begynner vi med [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), oppgir et datasett og estetisk mapping (med [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) og legger deretter til lag (som [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) for spredningsdiagrammer.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Er dette et nyttig diagram 🤷? Er det noe ved det som overrasker deg?

Det er ikke spesielt nyttig, ettersom alt det gjør er å vise dataene dine som en spredning av punkter i en gitt måned.
<br>


### **Hvordan gjør vi det nyttig?**

For å få diagrammer til å vise nyttige data, må du vanligvis gruppere dataene på en eller annen måte. For eksempel, i vårt tilfelle, vil det å finne gjennomsnittsprisen på gresskar for hver måned gi mer innsikt i de underliggende mønstrene i dataene våre. Dette leder oss til enda en **dplyr** gjennomgang:

#### `dplyr::group_by() %>% summarize()`

Gruppert aggregering i R kan enkelt beregnes ved hjelp av

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` endrer analyseenheten fra hele datasettet til individuelle grupper, som for eksempel per måned.

-   `dplyr::summarize()` lager en ny dataramme med én kolonne for hver grupperingsvariabel og én kolonne for hver av de oppsummeringsstatistikkene du har spesifisert.

For eksempel kan vi bruke `dplyr::group_by() %>% summarize()` til å gruppere gresskar i grupper basert på **Month**-kolonnen og deretter finne **gjennomsnittsprisen** for hver måned.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Kompakt!✨

Kategoriske variabler som måneder blir bedre representert ved hjelp av et stolpediagram 📊. Lagene som brukes for stolpediagrammer er `geom_bar()` og `geom_col()`. Sjekk `?geom_bar` for å lære mer.

La oss lage et!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩Dette er en mer nyttig datavisualisering! Det ser ut til å indikere at den høyeste prisen for gresskar forekommer i september og oktober. Stemmer dette med dine forventninger? Hvorfor eller hvorfor ikke?

Gratulerer med å ha fullført den andre leksjonen 👏! Du forberedte dataene dine for modellbygging, og deretter avdekket du flere innsikter ved hjelp av visualiseringer!



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
